In [ ]:
import os
import time
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl
import shutil
import sys
import pandas as pd

import flopy
from modflowapi import ModflowApi
from modflowapi.extensions import ApiSimulation

from bmi.wrapper import BMIWrapper

import pyswmm
from pyswmm import Simulation, Nodes
from pyswmm import Output

c:\Users\bbayrakt\AppData\Local\miniforge3\envs\liss\lib\site-packages\bmi\__init__.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
c:\Users\bbayrakt\AppData\Local\miniforge3\envs\liss\lib\site-packages\pkg_resources\__init__.py:2832: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
c:\Users\bbayrakt\AppData\Local\miniforge3\envs\liss\lib\site-packages\pkg_resources\__init__.py:2832: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap.responses')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/e

In [2]:
sys.path.append("../common")
from liss_settings import \
    libmf6, \
    get_dflow_grid_name, get_dflow_dtuser, \
    get_modflow_coupling_tag, get_modflow_grid_name, \
    silent, verbosity, \
    print_path, print_value

In [3]:
control_path = pl.Path("../dflow-fm/coarse/tides_2018/base/FlowFM.mdu") # change this if using a different D-Flow FM control file
grid_name = get_dflow_grid_name(control_path)
print(grid_name)

LIS_GPT_PJ_cells3_net


In [4]:
# swmm_path = pl.Path("../swmm/greenport/greenport_detailedsewer_v4_nosub175.inp").resolve()
# print(swmm_path, swmm_path.is_file())

In [5]:
# fpth = swmm_path.with_suffix(".out")
# if fpth.is_file():
#     print(f"removing: '{fpth}'")
#     fpth.unlink()
# fpth = swmm_path.with_suffix(".rpt")
# if fpth.is_file():
#     print(f"removing: '{fpth}'")
#     fpth.unlink()

In [6]:
# mf_grid_name = get_modflow_grid_name()
# print(mf_grid_name)

mf_grid_name = 'PJmf6'

In [7]:
dflowfm_dtuser = get_dflow_dtuser(control_path)
print(dflowfm_dtuser)

300.0


#### Set unit conversion factors

In [8]:
d2sec = 24. * 60. * 60.
hrs2sec = 60. * 60. 
m2ft = 3.28081
cfd2cms = 1.0 / ((m2ft**3) * 86400.)

#### Set the MODFLOW coupling frequency

Change the `mf_couple_freq_hours` value. Only tested for multiple of the D-Flow FM DtUser variable. Will not work for `mf_couple_freq_hours` values greater than 24.

In [9]:
mf_couple_freq_hours = 12  #Change this value to change the coupling frequency
mf_couple_freq = mf_couple_freq_hours * hrs2sec
dflow_per_mf = int(mf_couple_freq / dflowfm_dtuser)

In [10]:
print(f"MODFLOW coupling frequency {mf_couple_freq_hours} hours\nMODFLOW coupled to D-FLOW FM every {dflow_per_mf} output time step ({dflowfm_dtuser} sec.)") 

MODFLOW coupling frequency 12 hours
MODFLOW coupled to D-FLOW FM every 144 output time step (300.0 sec.)


In [11]:
mf_tag = get_modflow_coupling_tag(mf_couple_freq_hours)
print(f"MODFLOW coupling tag: {mf_tag}")

MODFLOW coupling tag: 12.00H


In [12]:
nstp = int(86400.0 / (dflow_per_mf * dflowfm_dtuser))
print(f"MODFLOW time steps per day: {nstp}")

MODFLOW time steps per day: 2


#### Set a few variables for controlling coupling

In [13]:
HDRY = -1e30
DEPTH_MIN = 0.1

#### Print the path of the modflow6 shared library

In [14]:
str(libmf6), libmf6.is_file()

('C:\\Users\\bbayrakt\\AppData\\Local\\miniforge3\\envs\\liss\\Scripts\\libmf6.dll',
 False)

#### Load the D-FLOW to MODFLOW weights

GHB weights

In [15]:
fpath = f"../mapping/PJ/dflow_{grid_name}_to_{mf_grid_name}_ghb.npz"
#fpath = f"../mapping/dflow_LIS_GPT_PJ_cells3_net_to_greenport_chd_ghb.npz"
npzfile = np.load(fpath)
print(fpath)
npzfile

../mapping/PJ/dflow_LIS_GPT_PJ_cells3_net_to_PJmf6_ghb.npz


NpzFile '../mapping/PJ/dflow_LIS_GPT_PJ_cells3_net_to_PJmf6_ghb.npz' with keys: dflow2mfghb, ghbmask, ghb2qext

In [16]:
dflow2mfghb = npzfile["dflow2mfghb"]
print(f'dflow2ghb shape: {dflow2mfghb.shape}')
ghbmask = npzfile["ghbmask"]
print(f'ghb mask shape :{ghbmask.shape}')
ghb2qext = npzfile["ghb2qext"]
print(f'ghb2qext shape: {ghb2qext.shape}')

dflow2ghb shape: (354, 21660)
ghb mask shape :(354,)
ghb2qext shape: (21660, 354)


CHD weights

In [17]:
fpath = f"../mapping/PJ/dflow_{grid_name}_to_{mf_grid_name}_chd.npz"
#fpath = f"../mapping/dflow_LIS_GPT_PJ_cells3_net_to_greenport_chd_chd.npz"
print(fpath)
npzfile = np.load(fpath)
npzfile

../mapping/PJ/dflow_LIS_GPT_PJ_cells3_net_to_PJmf6_chd.npz


NpzFile '../mapping/PJ/dflow_LIS_GPT_PJ_cells3_net_to_PJmf6_chd.npz' with keys: dflow2mfchd, chdmask, chd2qext

In [18]:
dflow2mfchd = npzfile["dflow2mfchd"]
print(f'dflow2chd shape: {dflow2mfchd.shape}')
chdmask = npzfile["chdmask"]
print(f'chd mask shape :{chdmask.shape}')
chd2qext = npzfile["chd2qext"]
print(f'chd2qext shape: {chd2qext.shape}')

dflow2chd shape: (1462, 21660)
chd mask shape :(1462,)
chd2qext shape: (21660, 1462)


#### Define paths for the model simulation

In [19]:
mf_base_path = pl.Path("../modflow/pj_2018_adjust_2CHDpkg/base/").resolve()
mf_run_path = pl.Path(f"../modflow/pj_2018_adjust_2CHDpkg/run_{mf_tag}/").resolve()

# mf_base_path = pl.Path("../modflow/greenport_chd/base/").resolve()
# mf_run_path = pl.Path(f"../modflow/greenport_chd/run_{mf_tag}/").resolve()

#### Load the base MODFLOW model

In [20]:
sim = flopy.mf6.MFSimulation.load(sim_ws=mf_base_path, verbosity_level=verbosity())
gwf = sim.get_model()

In [21]:
sim.set_sim_path(mf_run_path)
# create outputs in mf_run_path
(mf_run_path / 'outputs').mkdir(exist_ok=True)

#### Change TDIS to defined time steps

In [22]:
tdis = sim.get_package("TDIS")
perioddata = tdis.perioddata.array
print(perioddata)

[(365., 3, 1.3) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. ) (  1., 1, 1. )
 (  1., 1, 1. ) (  1., 1

In [23]:
perioddata["nstp"] = nstp
tdis.perioddata = perioddata
print(perioddata)

[(365., 2, 1.3) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. ) (  1., 2, 1. )
 (  1., 2, 1. ) (  1., 2

#### Write the new model files

In [24]:
sim.write_simulation(silent=silent())

#### Define base GHB variables

In [25]:
ghb_data0 = gwf.ghb.stress_period_data.get_dataframe()[0]
assert ghb_data0.shape[0] == ghbmask.shape[0]

#### Define base CHD variables

BNB note - seperate out the CHD packages? Need to ask Joe

2 CHD packages
1. Surface CHDs (chds in bay, and perimeter chd's in bay ,ALL in the top layer) which only has one external file
2. Perimeter CHDs (perimeter chds not in the bay, at all layers), 366 external files for every day. 

In [26]:
chd_all = gwf.get_package("chd_coast")
#chd_all = gwf.get_package("chd_surface")
chd_data0 = chd_all.stress_period_data.get_dataframe()[0]
print_value(chd_data0)
chd_data0
assert chd_data0.shape[0] == chdmask.shape[0]

In [27]:
chd_all.stress_period_data.get_dataframe()[0]

,cellid_layer,cellid_row,cellid_column,head,salinity,boundname
0,0,3,19,0.330,33.9,bay
1,0,3,20,0.330,34.6,bay
2,0,3,21,0.330,34.9,bay
3,0,3,22,0.330,34.7,bay
4,0,3,23,0.330,34.7,bay
...,...,...,...,...,...,...
1457,0,22,68,0.324,34.2,surface perimeter coastal
1458,0,24,70,0.324,34.4,surface perimeter coastal
1459,0,26,72,0.324,34.4,surface perimeter coastal
1460,0,28,74,0.324,34.4,surface perimeter coastal


#### SWMM data

In [28]:
# SWMM mappings
junctions = (
     "9",
     "10",
     "47",
     "54",
     "44",
     "34",
     "81",
     "155",
     "67",
     "69",
     "30",
     "118",
)

# these are the junction elevations in the SWMM inp file
swmm_inverts = {
    "9": 5,
    "10": 2,
    "47": 2.2,
    "54": 0.7,
    "44": 2.5,
    "34": 6.0,
    "81": 0.7,
    "155": 3,
    "67": -1.0,
    "69": 1.0,
    "30":  5.5,
    "118": 4.6,
 }

# MODFLOW -----------------
# map of the junction names and the MODFLOW (row, column)
# containing the junction
mf6_cells = {
    "9": (25, 32),
    "10": (24, 32),
    "47": (28, 33),
    "54": (29, 33),
    "44": (27, 33),
    "34": (28, 31),
    "81": (28, 30),
    "155": (28, 29),
    "67": (31, 32),
    "69": (31, 31),
    "30": (27, 31),
    "118": (35, 26),
}

#### Setup and initialize D-FLOW FM

You will need to set `dflow_dirpath` to the correct directory on your machine.

In [29]:
# dflow_root_path = pl.Path(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM").resolve()
# dll_paths = (
#     r"plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin",
#     r"plugins\DeltaShell.Dimr\kernels\x64\share\bin",
# )

In [30]:
# # os.environ["PATH"] = (
# #     str(dflow_dirpath) + os.pathsep + os.environ["PATH"]
# # )
# old_path = os.environ["PATH"]
# os.environ["PATH"] = ""
# path = ""
# for p in dll_paths:
#     path += f"{dflow_root_path / p}" + os.pathsep
# os.environ["PATH"] = path + old_path

In [31]:
# dflow_dirpath = dflow_root_path / f"{dll_paths[0]}"

In [32]:
# dflow_dirpath = os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin")
# dflow_deps_dirpath = (
#     os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\share\bin"),
# )
dflow_dirpath = pl.Path(r"..\dflow-fm\dflowfm_dll").resolve()
dflow_base = pl.Path(r"..\dflow-fm\coarse\tides_2018\base").resolve()
dflow_working = pl.Path(r"..\dflow-fm\coarse\tides_2018\run").resolve()
dflow_config = dflow_working / "FlowFM.mdu"
print(dflow_config)

D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\dflow-fm\coarse\tides_2018\run\FlowFM.mdu


In [33]:
if dflow_working.is_dir():
    shutil.rmtree(dflow_working)
shutil.copytree(dflow_base, dflow_working)
(dflow_working / "output").mkdir(parents=True, exist_ok=True)

In [34]:
# Add dflowfm dll folder to PATH so that it can be found by the BMIWrapper
os.environ["PATH"] = (
    str(dflow_dirpath) + os.pathsep + os.environ["PATH"]
)

In [35]:
(pl.Path(dflow_dirpath) / "dflowfm.dll").is_file()

True

#### Initialize D-Flow FM

In [36]:
dflowfm = BMIWrapper(
    engine="dflowfm",
    configfile=str(dflow_config),
)

In [37]:
import os
print("cwd:", os.getcwd())

cwd: d:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\notebooks-PJ


In [38]:
dflowfm.initialize()
# this line of code is what changes the cwd to  '\dflow-fm\coarse\tides\run'

In [39]:
import os
print("cwd:", os.getcwd())


cwd: D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\dflow-fm\coarse\tides_2018\run


#### Get data from D-FLOW FM

In [40]:
ndxi = int(dflowfm.get_var("ndxi"))
ndx = int(dflowfm.get_var("ndx")) # number of nodes
x = dflowfm.get_var("xz")
y = dflowfm.get_var("yz")
z = dflowfm.get_var("bl")
xy = [(xx, yy) for (xx, yy) in zip(x, y)]

ndx, ndxi, x.shape, y.shape # bnb note: are these values ok? answer: they shouldnt matter because they run with the gp model 

(21755, 21660, (21755,), (21755,))

In [41]:
dflowfm.get_var("s1").shape, dflowfm.get_var("s1")

((21755,),
 array([5.25852734, 5.77415958, 2.66252982, ..., 0.32488128, 0.34024957,
        0.3374276 ]))

In [42]:
dflowfm.get_var("hs").shape, dflowfm.get_var("hs")


((21755,),
 array([ 0.        ,  0.        ,  0.        , ..., 80.27836337,
        67.38442735, 70.16704295]))

In [43]:
qext = np.zeros(ndx)
qext.shape, qext

((21755,), array([0., 0., 0., ..., 0., 0., 0.]))

In [44]:
# dflowfm.set_var("qext", qext)

In [45]:
qext_cum = np.zeros(ndx)
qext_cum.shape

(21755,)

In [46]:
vextcum = dflowfm.get_var("vextcum")
vextcum.shape, vextcum

((21755,), array([0., 0., 0., ..., 0., 0., 0.]))

#### Initialize SWMM

In [47]:
# swmm_sim = pyswmm.Simulation(str(swmm_path))

# # build the swmm_nodes dictionary where the values are the swmm node objects
# # for example, "J1 = pyswmm.Nodes(swmm_sim)["J1"]"" , where J1 is the key
# # and the value is pyswmm.Nodes(swmm_sim)["J1"]
# swmm_nodes = {}
# for j in junctions:
#     swmm_nodes[j] = pyswmm.Nodes(swmm_sim)[j]

In [48]:
# swmm_sim.start()

#### Initialize MODFLOW using MODFLOW API

In [49]:
# set the mdll_path to be the absolute path where the mf6 dll is located
#mdll_path = pl.Path(r"D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\modflow\mf6dll\libmf6.dll")
mdll_path = str(r"D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\modflow\mf6dll\libmf6.dll")
print(mdll_path)

mf_run_path = str(mf_run_path)
print(mf_run_path)

D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\modflow\mf6dll\libmf6.dll
D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\modflow\pj_2018_adjust_2CHDpkg\run_12.00H


In [50]:
mf6 = ModflowApi(mdll_path, working_directory=mf_run_path)

In [51]:
mf6.initialize()

##### use recent modflowapi functionality for accessing data from the api in flopy-like data structures

In [52]:
# apisim = ApiSimulation.load(mf6)
# apiml = apisim.get_model()

# sewer_flow = apiml.get_package("SWMM")
# swmm_dtype = [("nodelist", "O"), ("q", float)]

#### Define MODFLOW variable tags and set pointer to MODFLOW variables

In [53]:
ghb_bhead_tag = mf6.get_var_address("BHEAD", "GWF", "GHB")
ghb_cond_tag = mf6.get_var_address("COND", "GWF", "GHB")
ghb_flow_tag = mf6.get_var_address("SIMVALS", "GWF", "GHB")

In [54]:
ghb_bhead_ptr = mf6.get_value_ptr(ghb_bhead_tag)
ghb_cond_ptr = mf6.get_value_ptr(ghb_cond_tag)
ghb_flow = np.zeros(ghb_bhead_ptr.shape)

In [55]:
# chd_head_tag = mf6.get_var_address("HEAD", "GWF", "CHD_SURFACE")
# chd_flow_tag = mf6.get_var_address("SIMVALS", "GWF", "CHD_SURFACE")

chd_head_tag = mf6.get_var_address("HEAD", "GWF", "CHD_coast")
chd_flow_tag = mf6.get_var_address("SIMVALS", "GWF", "CHD_coast")

In [56]:
chd_head_ptr = mf6.get_value_ptr(chd_head_tag)
chd_flow = np.zeros(chd_head_ptr.shape)

#### Create dictionaries for saving modified GHB data

In [57]:
ghb_elev_dict = {}
ghb_cond_dict = {}
chd_elev_dict = {}
qext_dict = {}
swmm_q_dict = {}

#### Function to update MODFLOW GHB data

In [58]:
# # TEST CELL
# # what does CHD head and GHB head and GHB cond look like?
# dflowfm.update()
# s = dflowfm.get_var("s1")[:ndxi] # water level
# print(s.shape,s.max(), s.min(), s.mean())

# has_nan = np.isnan(s).any()
# print(has_nan)

# d = dflowfm.get_var("hs")[:ndxi]
# print(d.shape,d.max(), d.min(), d.mean())

# has_nan = np.isnan(d).any()
# print(has_nan)



In [59]:

# print('running update_mf')
# mask = d == 0.0
# s[mask] = 0.0
# mult = np.full(d.shape, 1.0)
# mult[mask] = 0.0

# print('ghb_head')

# ghb_head = ghb_data0["bhead"].to_numpy()
# ghb_head[ghbmask] = dflow2mfghb.dot(s)[ghbmask] * m2ft
# ghb_bhead_ptr[:] = ghb_head[:] # does not work

# ghb_cond = ghb_data0["cond"].to_numpy()
# ghb_cond[ghbmask] = np.round(ghb_cond[ghbmask] * dflow2mfghb.dot(mult)[ghbmask],3)

# chd_head = chd_data0["head"].to_numpy()
# chd_head[chdmask] = np.round(dflow2mfchd.dot(s)[chdmask] * m2ft,3) 
# chd_head_ptr[:] = chd_head[:] # issue here and probably also in GHB_ptr
# # chd_head_ptr[:] = chd_head_ptr[:] * 1.5 #this works

# # # update results dictionary
# # ghb_elev_dict[key] = ghb_head.copy()
# # ghb_cond_dict[key] = ghb_cond.copy()
# # chd_elev_dict[key] = chd_head.copy()
# # print('done')

In [60]:
# chd_head.max()
# chd_head.min()
# chd_head.mean()

problem is likely because heads are below ghb + chd cells

find which ghb+chd cells have a botm01 elevation lower 0 
    - exclude cells from coupling? 
    or 
    -raise heads to be above those cells

In [61]:
def update_mf(key, s, d):
    print('running update_mf')
    mask = d == 0.0
    s[mask] = 0.0
    mult = np.full(d.shape, 1.0)
    mult[mask] = 0.0

    print('ghb_head')

    ghb_head = ghb_data0["bhead"].to_numpy()
    # print(f'ghb_data0 : {ghb_data0["bhead"].shape}') 
    # print(f'ghb_head :{ghb_head.shape}')

    ghb_head[ghbmask] = dflow2mfghb.dot(s)[ghbmask] * m2ft
    # print(f'ghbmask : {ghbmask.shape} , dflow2mfghb :{dflow2mfghb.shape}')

    ghb_cond = ghb_data0["cond"].to_numpy()
    print(f'ghb_cond : {ghb_data0["cond"].shape}') 

    ghb_cond[ghbmask] = ghb_cond[ghbmask] * dflow2mfghb.dot(mult)[ghbmask]
    
    print('update_mf, 4')
    ghb_bhead_ptr[:] = ghb_head[:] # does not work
    #ghb_bhead_ptr[:] = ghb_bhead_ptr[:] * 1.5 # _ptr variable is a pointer for the APR. 

    print('update_mf, 5')
    ghb_cond_ptr[:] = ghb_cond[:] #does not work
    #ghb_cond_ptr[:] = ghb_cond_ptr[:]
    
    print('update_mf, 6')
    chd_head = chd_data0["head"].to_numpy()
    print('update_mf, 7')
    chd_head[chdmask] = dflow2mfchd.dot(s)[chdmask] * m2ft
    print('update_mf, 8')
    
    chd_head_ptr[:] = chd_head[:] # does not work
    #chd_head_ptr[:] = chd_head_ptr[:] * 1.5 #this works
   
    # update results dictionary
    ghb_elev_dict[key] = ghb_head.copy()
    ghb_cond_dict[key] = ghb_cond.copy()
    chd_elev_dict[key] = chd_head.copy()
    print('done')

#### Function to update D-Flow FM Qext data

In [62]:
def update_dflow(key, d):
    ghb_flow = -mf6.get_value(ghb_flow_tag) * cfd2cms
    print(f'ghb_flow: {ghb_flow.shape, mf6.get_value(ghb_flow_tag).shape}')
    
    dflow_qext_ghb = ghb2qext.dot(ghb_flow)
    print(f'dflow_qext_ghb{dflow_qext_ghb.shape, ghb2qext.shape, ghb_flow.shape}')

    dflow_qext_ghb[d == 0.0] = 0.0
    
    chd_flow = -mf6.get_value(chd_flow_tag) * cfd2cms
    print(f'chd_flow{chd_flow.shape, mf6.get_value(chd_flow_tag).shape}')

    dflow_qext_chd = chd2qext.dot(chd_flow)

    dflow_qext_chd[d == 0.0] = 0.0

    dflow_qext = dflow_qext_ghb + dflow_qext_chd
    
    qext_cum[:ndxi] += dflow_qext[:ndxi]
    qext[:ndxi] = dflow_qext[:ndxi]
    dflowfm.set_var("qext", qext)

    # update results dictionaries
    qext_dict[key] = qext[:ndxi].copy()

#### Function to update SWMM and the MODFLOW SWMM well

In [63]:
# def update_swmm(key):
#     heads = apiml.X
#     mf6_spd = []
#     for key, value in swmm_nodes.items():
#         row, col = mf6_cells[key]
#         head = heads[0, row, col]
#         # might be nice to use the simulated swmm water-level
#         # in the junctions instead of the invert elevation
#         pot = head - swmm_inverts[key] * m2ft
#         if pot > 0.0:
#             Q = pot * 0.001
#         else:
#             Q = pot * 0.0002
#         value.generated_inflow(Q * cfd2cms)  # convert to CMS
#         mf6_spd.append(((0, row, col), -Q)) # unit conversion needed

#     # update gwf_swmm well file with new flux data - will be used next time step
#     mf6_spd = np.array(mf6_spd, dtype=swmm_dtype)
#     sewer_flow.stress_period_data.values = mf6_spd

#     # update results dictionary
#     swmm_q_dict[key] = mf6_spd["q"].copy()


#### Run each time step

In [64]:
# print(
#     f"DFLOWFM current_time: {dflowfm.get_current_time():15,.1f} sec. ({dflowfm.get_current_time()/86400.:15,.1f} days)\n"
#      + f"DFLOWFM end_time:     {dflowfm.get_end_time():15,.1f} sec. ({dflowfm.get_end_time()/86400.:15,.1f} days)"
# )

In [ ]:
mf6_start_date = datetime.datetime(2010,1,1)
mf6_end_date = mf6_start_date + datetime.timedelta(days=mf6.get_end_time())
dflow_start_date = datetime.datetime(2010,2,15)
dflow_end_date = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_end_time())
swmm_start_date = datetime.datetime(2010,1,1)

In [ ]:
idx = 0
jdx = 0
t0 = time.perf_counter()

# NOTE: we'll assume that the modflow model will run the longest
start_date = mf6_start_date
current_date = mf6_start_date

if mf6_start_date < dflow_start_date:
    # start mf6 and run until the start of the other two models
    while current_date <= dflow_start_date:
        mf6.update()
        current_date = start_date + datetime.timedelta(days=mf6.get_current_time())
    
while current_date <= dflow_end_date:
    print('now running mf and dflow together')
    idx += 1
    frac_comp = 1 - (dflow_end_date - current_date).days / (dflow_end_date - start_date).days
    print(f"(Current date: {current_date}) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
    dflowfm.update()
    current_date = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())

    if idx == int(dflow_per_mf):
        print(f"(Current date: {current_date}) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
        s = dflowfm.get_var("s1")[:ndxi] # water level
        d = dflowfm.get_var("hs")[:ndxi]
        
        mf6.prepare_time_step(mf6.get_time_step())
        update_mf(str(jdx), s, d)
        mf6.do_time_step()
        mf6.finalize_time_step()
        update_dflow(str(jdx), d)
        
        # # advance SWMM
        # update_swmm(str(jdx))
        # dt_sec = mf6.get_time_step() * d2sec
        # swmm_sim.step_advance(int(dt_sec))
        # try:
        #     swmm_sim.__next__()  
        # except StopIteration:
        #     break        

        # update counters
        idx = 0
        jdx += 1
    
    if current_date >= dflow_end_date:
        break

vextcum = dflowfm.get_var("vextcum")

t1 = time.perf_counter()
print(f"\nrun time: {(t1 - t0) / 60.} min")

31536000.0
s: [5.25852734 5.77415958 2.66252982 ... 0.60615411 0.60731516 0.60779785]4) ***
d[0.         0.         0.         ... 3.65320294 5.80294353 3.84093676]
step0
0.0
step1
running update_mf
ghb_head
ghb_cond : (354,)
update_mf, 4
update_mf, 5
update_mf, 6
update_mf, 7
update_mf, 8
done
step2
step3
step4
ghb_flow: ((354,), (354,))
dflow_qext_ghb((21660,), (21660, 354), (354,))
chd_flow((1462,), (1462,))
s: [ 0.          0.          0.         ... -0.31510788 -0.3115678te - (144) ***
 -0.31099167]
d[0.         0.         0.         ... 2.73194095 4.88406056 2.92214724]
step0
158.69565217391303
step1
running update_mf
ghb_head
ghb_cond : (354,)
update_mf, 4
update_mf, 5
update_mf, 6
update_mf, 7
update_mf, 8
done
step2
step3
step4
ghb_flow: ((354,), (354,))
dflow_qext_ghb((21660,), (21660, 354), (354,))
chd_flow((1462,), (1462,))
s: [ 0.          0.          0.         ... -0.32228847 -0.31748516e - (144) ***
 -0.3168285 ]
d[0.         0.         0.         ... 2.72476036 4.87814

#### Finalize models

In [ ]:
mf6.finalize()

In [ ]:
# swmm_sim.terminate_simulation()
# swmm_sim.report()
# swmm_sim.close()

In [ ]:
dflowfm.finalize()

#### Save ghb elevation and conductance data to compressed files

In [ ]:
np.savez_compressed(f"{mf_run_path}/ghb_elev.npz", **ghb_elev_dict)
np.savez_compressed(f"{mf_run_path}/ghb_cond.npz", **ghb_cond_dict)

#### Save chd elevation to compressed file

In [ ]:
np.savez_compressed(f"{mf_run_path}/chd_elev.npz", **chd_elev_dict)

#### Save qext data to compressed file

In [ ]:
np.savez_compressed(f"{mf_run_path}/qext.npz", **qext_dict)

#### Save SWMM flux data to compressed file

In [ ]:
# np.savez_compressed(f"{mf_run_path}/swmm_q.npz", **swmm_q_dict)